Tutorial em texto para os inscritos no meu curso de Machine Learning na Udemy sobre a classificação de textos utilizando a biblioteca Scikit-learn.

O objetivo é treinar um algoritmo para classificar corretamente o tema de notícias com base no texto escrito.

Iremos utilizar um conjunto de dados com milhares de notícias em inglês e treinar o algoritmo para indentificar se a notícia é sobre "Ateímos", "Cristianismo", "Computação Gráfica" ou "Ciências Médicas".

In [1]:
categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']

In [2]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

**Pré-processamento dos dados (Pre-processing)**

Quando trabalhamos com textos, precisamos realizar algumas técnicas de pré-processamento dos dados para extrair algumas *features* importantes.

O primeiro passo é construir um dicionário usando *CountVectorizer* que cria uma contagem de ocorrência de cada palavra nos textos. Quando maior o índice, mais ocorrência ocorre desta palavra, veja:

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)

print("Muita frequência do termo 'of' -> "+str(count_vect.vocabulary_.get(u'of')))
print("Frequência menor do termo 'algorithm' -> "+str(count_vect.vocabulary_.get(u'algorithm')))


Muita frequência do termo 'of' -> 23610
Frequência menor do termo 'algorithm' -> 4690


Para evitar que textos longos, onde ocorrem mais frequências de palavras, tenham um resultado diferente de textos curtos, por mais que ambos falem sobre o mesmo tema, é necessário criar uma outra *feature* para capturar isso. Chamamos essa *feature* de **Term Frequencies** ou simplesmente **tf**, onde dividimos o número de ocorrências de cada palavra pelo total de palavras no documento.

Realizamos mais uma transformação chamada de **idf** ou **Inverse Document Frequency**, que diminui o peso de palavras que se repetem muitas vezes entre os diferentes textos (exemplo: of, the, are, you, etc..), por serem menos informativas e não ajudarem na análise.

Ambas as transformações são realizadas em uma mesma função chamada *fit_transform*:

In [4]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

**Treinando o modelo**

Em alguns materiais de machine learning é comum encontrar o nome *classifier* como sinonimo de *model*, ou seja, como nome para referenciar o algoritmo de machine learning utilizado.

Neste caso, utilizaremos o classifier *Multinomial Naive Bayes*:

In [5]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

Vamos utilizar o algoritmo para tentar classificar duas frases "God is love" e "OpenGL on the GPU is fast". Sendo a primeira relacionada a categoria "Cristianismo" e a segunda a catergoria "Computação Gráfica".

In [6]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


Conseguimos ver que o algoritmo identificou corretamente a categoria das duas frases!

Podemos utilizar neste exemplo uma técnica muito utilizada em Machine Learning chamada Pipeline. Que é simplesmente colocar as diversas transformações dos dados e o próprio algoritmo em uma mesma função, sem precisar realizar o pre-processing e a escolha do algoritmo em separado. Basicamente funciona como uma **sequência** de tarefas que serão realiza com os dados:


In [7]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),])

text_clf.fit(twenty_train.data, twenty_train.target)  

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

Podemos calcular facilmente a precisão do modelo:

In [8]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test',
    categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.8348868175765646

Nosso modelo conseguiu prever com **83%** de precisão o tema dos textos dos dados de teste corretamente!

Podemos alterar o *classifier* para ver se o resultado melhora. Vamos testar com um algoritmo de *support vector machine* (SVM), mais precisamente o *SGDClassifier*.

In [9]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
     ('vect', CountVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                           alpha=1e-3, random_state=42,
                           max_iter=5, tol=None)),])

text_clf.fit(twenty_train.data, twenty_train.target)  
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)    


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


0.9127829560585885

A precisão aumentou bastante! Para **91,27%**!

Neste exemplo prático podemos utilizar também a técnica de *GridSearch*, para encontrar os melhores Hyperparâmetros para este modelo.

Bascimente vamos rodar o modelo diversas vezes trocando os parâmetros do pipeline e descobrir os melhores:

In [10]:
from sklearn.model_selection import GridSearchCV
parameters = {
     'vect__ngram_range': [(1, 1), (1, 2)],
     'tfidf__use_idf': (True, False),
     'clf__alpha': (1e-2, 1e-3),}

gs_clf = GridSearchCV(text_clf, parameters, cv=5, iid=False, n_jobs=-1)
gs_clf = gs_clf.fit(twenty_train.data[:400], twenty_train.target[:400])

print(gs_clf.best_score_)

for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

0.9151349867929058
clf__alpha: 0.001
tfidf__use_idf: True
vect__ngram_range: (1, 2)


/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


Conseguimos melhorar ainda mais o modelo! Utilizando os hyperparâmetros alpha: 0.001 com transformação de idf e vetorização com escala (1, 2), o modelo subiu sua precisão para **91,51%**.